In [1]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tweepy
import requests
import time
from datetime import date
import json
import sys

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Define location for config file
sys.path.append('../../..')

# Import Twitter API keys.
from config import consumer_key, consumer_secret, access_token, access_token_secret, twitter_api_sn

In [2]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Note: Twitter sometimes did not recognize/log mentions when requests were made using '@jxavie analyze: @target_user'
# As a work around, a for loop was used to aggregate user mentions into a list, before using another for loop to 
# determine the target account by comparing the contents of the user mentions list to the screen name associated 
# with the API keys.  While this allows more flexibility in the syntax, requests are still limited one target account
# and requires the inclusion of the word 'analyze'.

# Define function to retrieve mentions and determine if sentiment analysis is requested
def retrieve_mentions():
    
    # Initialize list to store mentions within a tweet
    mentions = []
    
    # Request tweets containing mentions of screen name associated with API key
    tweets = api.mentions_timeline()
    
    # Define tasks to perform if record of mentions found 
    try:

        # Return latest tweet with mention
        request = tweets[0]['text']

        # Set all tweet characters to lower case
        request_lower = request.lower()

        # Determine tweet ID
        #tweet_id = request[0]['id']

        # Determine user making request for analysis
        tweeter = tweets[0]['user']['screen_name']

        # Use conditional to determine whether an analysis is requested
        if (request_lower.find('analyze') != -1):

            # Use for loop to determine users mentioned in tweet and append user handles to mentions list
            for i in range(len(tweets[0]['entities']['user_mentions'])):
                mentions.append(tweets[0]['entities']['user_mentions'][i]['screen_name'])

            # Use for loop and condition to determine user to be analyzed
            for i in range(len(mentions)):
                if twitter_api_sn != mentions[i]:
                    target_user = f'@{mentions[i]}'

        # Set target_user value to null if no analysis is requested and print error message
        else:
            target_user = None
            print('No analysis request found.')
    
    # Set return variables to none if no mentions record found and print error message
    except:
        target_user = None
        tweeter = None
        print("No tweets requesting analysis found.")
    
    # Define variables to return
    return target_user, tweeter

In [4]:
# Define function to retrieve tweets from target account and store in a dataframe
def retrieve_tweets(handle):
    
    # Initialize lists to store tweets and tweet information
    user_sn = []
    tweet_texts = []
    tweet_time = []
    
    # Initialize lists for sentiments
    compound_list = []
    positive_list = []
    neutral_list = []
    negative_list = []
    
    # Loop through 1 pages of tweets for testing
    # for x in range(1,2):
    
    # Loop through 25 pages of tweets (for a total of 500 tweets)
    for x in range(1,26):
        
        # Retrieve tweets from home feed
        public_tweets = api.user_timeline(handle, page=x, tweet_mode='extended')
        
        # Loop through tweets and store retrieved information in list
        for tweet in public_tweets:
            
            # Store tweets and tweet information in appropriate lists
            user_sn.append(tweet['user']['screen_name'])
            tweet_texts.append(tweet['full_text'])
            tweet_time.append(tweet['created_at'])
            
            #Run Vader Analysis on tweet
            results = analyzer.polarity_scores(tweet['full_text'])
            
            # Add each value to the appropriate list
            compound_list.append(results['compound'])
            positive_list.append(results['pos'])
            neutral_list.append(results['neu'])
            negative_list.append(results['neg'])
    
    # Store tweets as an entry in a dictionary
    tweets_df = pd.DataFrame([user_sn,tweet_texts,tweet_time,compound_list,positive_list,neutral_list,negative_list]).transpose()
    tweets_df.columns = ['source','tweet','timestamp','compound','positive','neutral','negative']
    
    # Add tweet numbering
    tweets_df = tweets_df.reset_index()
    tweets_df['index'] = tweets_df['index'] + 1
    tweets_df = tweets_df.rename(columns={'index':'tweet number'})
    
    # Convert unix timestamps to dates
    tweets_df['timestamp'] = pd.to_datetime(tweets_df['timestamp'])

    # Cast sentiment values as float
    tweets_df['compound'] = tweets_df['compound'].astype('float')
    tweets_df['positive'] = tweets_df['compound'].astype('float')
    tweets_df['neutral'] = tweets_df['compound'].astype('float')
    tweets_df['negative'] = tweets_df['compound'].astype('float')

    # Create new column with formatted date
    # news_df['date'] = news_df['timestamp'].map(lambda x: x.strftime('%m/%d/%y'))

    # Truncate and Reorder columns of dataframe
    tweets_df = tweets_df[['source','tweet number','tweet','timestamp','compound','positive','neutral','negative']]
    
    # Save dataframe as csv
    tweets_df.to_csv(f'Output/{handle}_tweets.csv',index=False)
    
    # Print success message
    print(f'Retrieved and analyzed 500 most recent tweets from {handle}.')
    
    return tweets_df

In [5]:
# Define function to create plot
def create_plot(handle, dataframe):

    # Retrieve date
    date_analysis = date.today().strftime("%m/%d/%y")

    # Specify size of plot
    plt.rcParams['figure.dpi'] = 200

    # Generate plot
    fig, tweets_plot = plt.subplots()

    # Define scatter plot data and settings for markers
    tweets_plot.plot(dataframe['tweet number'], dataframe['compound'], label=handle, marker='o', markersize=4, linewidth=0.3) 

    # Define labels and formatting of plot
    plt.title(f'Sentiment Analysis of Tweets ({date_analysis})', fontsize='small')
    plt.xlabel('Tweets Ago', fontsize='x-small')
    plt.ylabel('Tweet Polarity', fontsize='x-small')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1), shadow=False, fontsize='xx-small', 
               title='Tweets', title_fontsize = 'x-small', frameon=False)

    # Define tick mark locations
    x_major_ticks = np.arange(500,-5,-100)
    y_major_ticks = np.arange(-1,1.5,0.5)

    # Set format for tick marks
    plt.xticks(fontsize='x-small')
    plt.yticks(fontsize='x-small')
    tweets_plot.set_xticks(x_major_ticks)
    tweets_plot.set_yticks(y_major_ticks)

    # Define min and max value limits for x- and y-axes
    plt.xlim(505, -5, 100)
    plt.ylim(-1.05, 1.05, 0.5)

    # Remove border around plot
    tweets_plot.spines['top'].set_visible(False)
    tweets_plot.spines['bottom'].set_visible(False)
    tweets_plot.spines['right'].set_visible(False)
    tweets_plot.spines['left'].set_visible(False)

    # Remove tick marks
    tweets_plot.tick_params(axis=u'both', which=u'both',length=0)

    # Show grid
    plt.grid(color='whitesmoke', alpha=1.5)

    # Set plot background colot
    tweets_plot.set_facecolor('lightgray')
    
    # Create variable to store figure name and location
    plot_image = f'Output/{handle}_sentiment_plot.png'
    
    # Save figure
    plt.savefig(plot_image,bbox_inches='tight')

    # Show figure
    # plt.show()

    # Print message
    print(f'Plot of Vader Analysis on tweets by {handle} generated.')
    
    return plot_image

In [6]:
# Define function to post status
def tweet_plot(plot, handle, requestor):
        
    # Create a status update
    # api.update_with_media("../Resources/too-much-big-data.jpg", "Testing...")
    api.update_with_media(plot, f'New Tweet Analysis: {handle} \n(Thanks @{requestor} for the request!)')
    
    print(f'Vader Analysis plot for {handle} posted on twitter. \n')

In [ ]:
# Initialize list to store users that have been analyzed
analyzed_users = []
# mention_id = []

# Run infinite loop
while(True):

    # Run retrieve_mentions function
    target_user, tweeter = retrieve_mentions()
    
    # Initialize counter variable
    counter = 0
    
    # Use conditional to determine if analysis is to be performed
    # Set analyze variable to True if analysis is to be performed; set to False otherwise 
    # If analysis is to be performed add target user handle to analyzed_users list
    # Compare target_user to values stored in analyzed_users list.
    # Analysis to be performed only if target_user has not already been analyzed
    if target_user == None:
        analyze = False
    else:
        if len(analyzed_users) == 0:
            analyze = True
            analyzed_users.append(target_user)
        else:
            for i in analyzed_users:
                if i == target_user:
                    counter += 1 
            if counter == 0:
                analyze = True
                analyzed_users.append(target_user)
            else:
                analyze = False
    
    # Run functions if target user is identified; print error message otherwise
    if analyze == True:
        print(f'User to analyze: {target_user}')
        tweets_df = retrieve_tweets(target_user)
        plot_image = create_plot(target_user, tweets_df)
        tweet_plot(plot_image, target_user, tweeter)
    else:
        print('No new analysis request found. \n')
    
    # Set timer to re-run code after 5 minutes
    time.sleep(5*60)

User to analyze: @TheSimpsons
Retrieved and analyzed 500 most recent tweets from @TheSimpsons.
Plot of Vader Analysis on tweets by @TheSimpsons generated.
Vader Analysis plot for @TheSimpsons posted on twitter. 

User to analyze: @HomerJSimpson
Retrieved and analyzed 500 most recent tweets from @HomerJSimpson.
Plot of Vader Analysis on tweets by @HomerJSimpson generated.
Vader Analysis plot for @HomerJSimpson posted on twitter. 

No new analysis request found. 

User to analyze: @TheOnion
Retrieved and analyzed 500 most recent tweets from @TheOnion.
Plot of Vader Analysis on tweets by @TheOnion generated.
Vader Analysis plot for @TheOnion posted on twitter. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No new analysis request found. 

No